# Schema.org Ontology Resource Templates

In [2]:
import datetime
import json
from collections import OrderedDict
import rdflib
import requests
from bs4 import BeautifulSoup
print(datetime.datetime.utcnow())

2019-06-12 18:22:26.327657


In [62]:
def init_profile(ident, title, description):
    profile = OrderedDict()
    profile["Profile"] = OrderedDict()
    profile["Profile"]["resourceTemplates"] = []
    profile["Profile"]["id"] = ident
    profile["Profile"]["date"] = datetime.datetime.utcnow().strftime("%Y-%m-%d")
    profile["Profile"]["description"] = description
    profile["Profile"]["author"] = "Jeremy Nelson"
    profile["Profile"]["title"] = title
    return profile

def init_resource_template(**kwargs):
    resource_template = OrderedDict()
    resource_template["id"] = kwargs.get("ident")
    resource_template["propertyTemplates"] = kwargs.get("propertyTemplates", [])
    resource_template["resourceURI"] = kwargs.get("uri")
    resource_template["resourceLabel"] = kwargs.get("label")
    return resource_template

In [19]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).text)
    
def property_template(tr):
    property_template = { "propertyURI": tr.attrs["resource"],
                          "mandatory": "false",
                          "repeatable": "true",
                          "type": "literal",
                          "valueConstraint": {
                            
                          }
                        }
    table_head = tr.find_all("th")
    table_data = tr.find_all("td")
    raw_label = table_head[0].text.strip()
    label = []
    for i, char in enumerate(raw_label):
        if i > 0 and char.isupper():
            label.append(f" {char}")
        else:
            label.append(char)
    property_template["propertyLabel"] = ''.join(label).title()
    if len(table_data) > 1:
        property_template["remark"] = table_data[1].text
    return property_template
   
    
    

In [2]:
with open('/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaThingProfile.json') as fo:
    thing = json.load(fo)

In [3]:
for key in thing.keys():
    print(key)

Profile


## Create Work Profile 2019-06-10

In [28]:
creative_work_profile = init_profile("profile:schema:creative_work", 
                                     "Schema.org Creative Work Profile", 
                                    """The most generic kind of creative work, including books, movies, 
                                    photographs, software programs, etc.""")
property_templates = [
   {
    "mandatory": "false",
    "propertyLabel": "Thing Properties",
    "propertyURI": "http://schema.org/Thing",
    "repeatable": "true",
    "type": "resource",
    "valueConstraint": {
      "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:Thing"
      ]
    }
  }
]
creative_work_resource_template = init_resource_template(ident="sinopia:resourceTemplate:schema:CreativeWork",
                                                         propertyTemplates=property_templates,
                                                         uri="http://schema.org/CreativeWork",
                                                         label="Creative Work")

creative_work_soup = get_soup("https://schema.org/CreativeWork")
def_table = creative_work_soup.find("table")
def_table_rows = def_table.find_all('tr')

In [29]:
for row in def_table_rows[2:]:
    try:
        creative_work_resource_template["propertyTemplates"].append(
            property_template(row))
    except:
        break

In [26]:
print(f"Length: {len(creative_work_resource_template['propertyTemplates'])}")

Length: 90


In [31]:
creative_work_profile["Profile"]["resourceTemplates"].append(creative_work_resource_template)

In [32]:
with open("/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaCreativeWork.json", "w+") as fo:
    json.dump(creative_work_profile, fo, indent=2, sort_keys=True)

## Book Profile 2019-06-10

In [83]:
book_profile = init_profile("sinopia:profile:schema:Book", "Schema.org Book Profile", "A Book")
property_templates = [
   {
    "mandatory": "false",
    "propertyLabel": "Thing Properties",
    "propertyURI": "http://schema.org/Thing",
    "repeatable": "true",
    "type": "resource",
    "valueConstraint": {
      "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:Thing"
      ]
    }
  },
  {
    "mandatory": "false",
    "propertyLabel": "CreativeWork Properties",
    "propertyURI": "http://schema.org/CreativeWork",
    "type": "resource",
    "valueConstraint": {
    "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:CreativeWork"
        ]
    }
  }
]
book_resource_template = init_resource_template(ident="sinopia", propertyTemplates=property_templates, uri="http://schema.org/Book", label="Book")
book_profile['Profile']['resourceTemplates'].append(book_resource_template)

In [84]:
book_soup = get_soup("https://schema.org/Book")
first_table = book_soup.find("table")

In [87]:
table_rows = first_table.find_all("tr")
for row in table_rows[2:8]:
    book_resource_template["propertyTemplates"].append(
        property_template(row)
    )

In [81]:
print(table_rows[2].find_all("td")[1].text)

Indicates whether the book is an abridged edition.


In [88]:
with open("/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaBookProfile.json", "w+") as fo:
    json.dump(book_profile, fo, indent=2, sort_keys=True)

## Person Profile 2019-06-12 &amp; 2019-06-13

In [69]:
person_profile = init_profile("sinopia:profile:schema:Person", 
                              "Schema.org Person Profile", 
                              "A person (alive, dead, undead, or fictional).")
property_templates = [
   {
    "mandatory": "false",
    "propertyLabel": "Thing Properties",
    "propertyURI": "http://schema.org/Thing",
    "repeatable": "true",
    "type": "resource",
    "valueConstraint": {
      "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:Thing"
      ]
    }
  }
]
person_rt = init_resource_template(
    ident="sinopia:resourceTemplate:schema:Person",
    propertyTemplates=property_templates, 
    uri="http://schema.org/Person",
    label="A person (alive, dead, undead, or fictional).")

In [38]:
person_soup = get_soup("https://schema.org/Person")
first_table = person_soup.find("table")
table_rows = first_table.find_all("tr")

In [73]:
len(person_rt["propertyTemplates"])

54

In [72]:
for row in table_rows[2:55]:
    try:
        person_rt["propertyTemplates"].append(property_template(row))
    except:
        break
    

In [74]:
person_profile["Profile"]['resourceTemplates']

[]

In [75]:
person_profile["Profile"]['resourceTemplates'].append(person_rt)

In [77]:
with open("/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaPersonProfile.json", "w+") as fo:
    json.dump(person_profile, fo, indent=2, sort_keys=True)